In [1]:
import pandas as pd
import os
import sys  
from cryptography.fernet import Fernet
from config import key

In [2]:
def encrypt(item, crypto, encoding='utf-8'):
    
    if pd.isnull(item):
        item = 'null'
    
    assert type(crypto) == Fernet
    
    item = bytes(item, encoding = encoding)
    
    return crypto.encrypt(item).decode(encoding)

In [3]:
def decrypt(item, crypto, encoding='utf-8'):
    
    bytes(item, encoding = encoding)
    bytes_decript = crypto.decrypt(item)
    
    return bytes_decript.decode('utf-8')

In [4]:
def get_files(path):
    
    files = [os.path.join(path, file)
            for file in os.listdir(path)]
    
    return files

In [5]:
def dados_inscricoes_audiencia():
    
    f_audiencia = get_files('../Dados_originais/Dados_inscricoes_audiencia/')
    dfs = []
    for f in f_audiencia:
        audiencia = f.split('/')[-1]
        df = pd.read_excel(f)
        df['Audiencia'] = audiencia
        dfs.append(df)
        
    return pd.concat(dfs)

In [6]:
def dados_banco_dados():
    
    files_site = get_files('../Dados_originais/Dados_Banco_de_Dados')
    
    contrib_f = [f for f in files_site if f.split('\\')[-1].startswith('contrib')][0]
    prop_f = [f for f in files_site if f.split('\\')[-1].startswith('result')][0]
    
    return pd.read_csv(contrib_f), pd.read_csv(prop_f)

In [7]:
def filtrar_e_renomear(df, nome_df):
    
    colunas = {
    'prop' : {'Autor' : 'nome', 'Email' : 'email'},
    'contrib' : {'Usuário' : 'nome', 'Email' : 'email'},
    'audi' : {'Nome:' : 'nome', 'E-mail:': 'email'}
    }
    
    df = df[colunas[nome_df].keys()].copy()
    df.rename(colunas[nome_df], axis = 1, inplace = True)
    
    return df

In [8]:
def juntar_tudo():
    
    audiencias = dados_inscricoes_audiencia()
    contrib, prop = dados_banco_dados()
    
    dfs = [
        filtrar_e_renomear(audiencias, 'audi'),
        filtrar_e_renomear(contrib, 'contrib'),
        filtrar_e_renomear(prop, 'prop')
    ]
    
    return pd.concat(dfs)
    
    

In [9]:
df = juntar_tudo()

In [10]:
df.email.duplicated().sum()

3092

In [11]:
df.duplicated().sum()

2917

In [12]:
df[df.email.duplicated(keep = False)].sort_values(by='email').to_excel('lista_nomes_email_duplicatas.xlsx')

In [13]:
deduplicado = df[~df['email'].duplicated()].copy()
deduplicado['nome'].duplicated().sum()

1761

In [14]:
deduplicado[deduplicado['nome'].duplicated(keep=False)].sort_values(by='email')

,nome,email
24,NaN,mgbove47@gmail.com
23,NaN,mariajosesousa@live.com
36,NaN,vandoirlima3@gmail.com
46,NaN,11968185275rafaella@gmail.com
25,NaN,1970lucianodossantos@gmail.com
...,...,...
37,NaN,zoehpares@gmail.com
44,NaN,zoioale@hotmail.com
86,NaN,Álvaro.fred.fernandes@gmail.com
50,NaN,élienerosaandrade @81gmail.com


In [15]:
deduplicado['nome'].isnull().sum() #VEM DA PLANILHA DE AUDIENCIAS

1759

In [16]:
crypto = Fernet(key)

In [17]:
deduplicado['nome_encript'] = deduplicado['nome'].apply(lambda x: encrypt(x, crypto))

In [19]:
deduplicado.to_excel('../Dados_gerados/lista_nomes_sem_duplicatas.xlsx')